In [ ]:
import numpy as np
import cv2 as cv
import os
import torch
from PIL import Image
import matplotlib.pyplot as plt
# tensorflow
import tensorflow.keras as keras
import tensorflow as tf
tf.keras.layers.RandomFlip
# image processing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
# model / neural network
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#mcp_Metacarpophalangeal joint
path = "/content/drive/MyDrive/MCP_arthritis"
  # List the files in the folder
import cv2
files = os.listdir(path)

# Read the first three images
RA_MCP=[]
RA_MCP_label=[]
for file in files:

  # Construct the full path to the file

  file_path = os.path.join(path, file)

# Read the image

  img = cv2.imread(file_path)
  img= Image.fromarray(img,"RGB")

  img= img.resize((224,224))
  img= np.array(img)
  RA_MCP.append(img)
  RA_MCP_label.append(0)

In [ ]:
base_dir = 'Arthritis'
os.mkdir(base_dir)
# create new folders inside base_dir
RAmcp = os.path.join(base_dir, 'MCP')
os.mkdir(RAmcp)
Normalpatient = os.path.join(base_dir, 'Normal')
os.mkdir(Normalpatient)

In [ ]:
import shutil
# Get a list of images in each of the two folders
folder_1 = os.listdir('/content/drive/MyDrive/MCP_arthritis')
folder_2 = os.listdir('/content/drive/MyDrive/normal MCP')
base_dir1='/content/Arthritis/MCP'
base_dir2='/content/Arthritis/Normal'

# Transfer the train images
for image in folder_1:
        # source path to image
        src = os.path.join('/content/drive/MyDrive/MCP_arthritis', image)
        # destination path to image
        dst = os.path.join(base_dir1, image)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

for image in folder_2:
        # source path to image
        src = os.path.join('/content/drive/MyDrive/normal MCP', image)
        # destination path to image
        dst = os.path.join(base_dir2, image)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [ ]:
import shutil
# Get a list of images in each of the two folders
folder_1 = os.listdir('/content/drive/MyDrive/MCP_arthritis')
folder_2 = os.listdir('/content/drive/MyDrive/normal MCP')
base_dir1 = '/content/augmented_images' # Create a new destination folder
base_dir2 = '/content/Mcpaugmented_images' # Create a new destination folder

os.makedirs(base_dir1, exist_ok=True) # Create the destination folders if they don't exist
os.makedirs(base_dir2, exist_ok=True)

# Transfer the train images
for image in folder_1:
    # source path to image
    src = os.path.join('/content/drive/MyDrive/MCP_arthritis', image)
    # destination path to image
    dst = os.path.join(base_dir1, image) # Copy to the new folder
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)

for image in folder_2:
    # source path to image
    src = os.path.join('/content/drive/MyDrive/normal MCP', image)
    # destination path to image
    dst = os.path.join(base_dir2, image) # Copy to the new folder
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)

In [ ]:
import imgaug.augmenters as iaa
import cv2
import os
import numpy as np
from glob import glob

# Create an output directory if it does not exist
output_dir = '/content/Arthritis/MCP'
#os.makedirs(output_dir, exist_ok=True)

# Define the augmentation sequence
aug = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    iaa.GaussianBlur(sigma=(0, 0.5)), # blur images with a sigma between 0 and 0.5
    iaa.LinearContrast((0.75, 1.5)), # improve or worsen the contrast
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)), # add gaussian noise
    iaa.Multiply((0.8, 1.2)), # make images darker or brighter
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate images
        rotate=(-25, 25), # rotate images
        shear=(-8, 8) # shear images
    )
], random_order=True) # apply augmenters in random order

# Load images
image_paths = glob('/content/Arthritis/MCP/*.jpg')  # Change the pattern according to your images
#images = [cv2.imread(img_path) for img_path in image_paths]
images = []
#img = cv2.imread(image_paths)
#print(img)
for img_path in image_paths:
    img = cv2.imread(img_path)
    if img is not None:  # Check if image loaded successfully
        images.append(img)
    else:
        print(f"Failed to load image: {img_path}")
# Ensure we have the correct number of images
num_generated_images = 0
target_num_images = 500
while num_generated_images < target_num_images:
    for idx, img in enumerate(images):
        if num_generated_images >= target_num_images:
            break
        augmented_image = aug(image=img)
        output_path = os.path.join(output_dir, f'augmented_{num_generated_images:04d}.jpg')
        cv2.imwrite(output_path, augmented_image)
        num_generated_images += 1

print(f"Generated {num_generated_images} augmented images.")

In [ ]:
import imgaug.augmenters as iaa
import cv2
import os
import numpy as np
from glob import glob

# Create an output directory if it does not exist
output_dir = '/content/Arthritis/Normal'
os.makedirs(output_dir, exist_ok=True)

# Define the augmentation sequence
aug = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    iaa.GaussianBlur(sigma=(0, 0.5)), # blur images with a sigma between 0 and 0.5
    iaa.LinearContrast((0.75, 1.5)), # improve or worsen the contrast
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)), # add gaussian noise
    iaa.Multiply((0.8, 1.2)), # make images darker or brighter
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate images
        rotate=(-25, 25), # rotate images
        shear=(-8, 8) # shear images
    )
], random_order=True) # apply augmenters in random order

# Load images
image_paths = glob('/content/Arthritis/Normal/*.jpg')  # Change the pattern according to your images
#images = [cv2.imread(img_path) for img_path in image_paths]
images = []
#img = cv2.imread(image_paths)
#print(img)
for img_path in image_paths:
    img = cv2.imread(img_path)
    if img is not None:  # Check if image loaded successfully
        images.append(img)
    else:
        print(f"Failed to load image: {img_path}")
# Ensure we have the correct number of images
num_generated_images = 0
target_num_images = 500
while num_generated_images < target_num_images:
    for idx, img in enumerate(images):
        if num_generated_images >= target_num_images:
            break
        augmented_image = aug(image=img)
        output_path = os.path.join(output_dir, f'augmented_{num_generated_images:04d}.jpg')
        cv2.imwrite(output_path, augmented_image)
        num_generated_images += 1

print(f"Generated {num_generated_images} augmented images.")

In [ ]:
!zip -r '/content/drive/MyDrive/New_Arthritis.zip' '/content/Arthritis'

In [ ]:
base_dir = 'Arthritis'
os.mkdir(base_dir)
# create new folders inside base_dir
mcp1 = os.path.join(base_dir, 'MCP')
os.mkdir(mcp1)
MCP2 = os.path.join(base_dir, 'NORMAL')
os.mkdir(MCP2)

In [ ]:
import os
import shutil

# Example paths (replace with your actual paths)
source_folder = '/content/augmented_images'
destination_folder = '/content/Arthritis/NORMAL/'

# Create destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Function to transfer images
def transfer_images(source_dir, dest_dir):
    # Iterate over files in source_dir
    for filename in os.listdir(source_dir):
        source_file = os.path.join(source_dir, filename)
        dest_file = os.path.join(dest_dir, filename)
        # Transfer the file from source to destination
        shutil.move(source_file, dest_file)

# Transfer images from source_folder to destination_folder
transfer_images(source_folder, destination_folder)

print("Images transferred from", source_folder, "to", destination_folder)

In [ ]:
import os
import shutil

# Example paths (replace with your actual paths)
source_folder = '/content/Mcpaugmented_images'
destination_folder = '/content/Arthritis/MCP'

# Create destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Function to transfer images
def transfer_images(source_dir, dest_dir):
    # Iterate over files in source_dir
    for filename in os.listdir(source_dir):
        source_file = os.path.join(source_dir, filename)
        dest_file = os.path.join(dest_dir, filename)
        # Transfer the file from source to destination
        shutil.move(source_file, dest_file)

# Transfer images from source_folder to destination_folder
transfer_images(source_folder, destination_folder)

print("Images transferred from", source_folder, "to", destination_folder)

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio('/content/drive/MyDrive/Arthritis', output='new_arthritis', seed=1337, ratio=(0.75, 0.1, 0.15))

In [ ]:
!zip -r '/content/drive/MyDrive/USG_arth.zip' '/content/new_arthritis'

In [ ]:
!unzip '/content/drive/MyDrive/USG_arth.zip'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

valid_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/new_arthritis/test/MCP',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/new_arthritis/train/MCP',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')
valid_set = test_datagen.flow_from_directory('/content/new_arthritis/val/MCP',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
pip install scikit-learn

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define directories
original_data_dir = '/content/new_arthritis/test/MCP'
train_dir = 'data/train'
val_dir = 'data/val'
test_dir = 'data/test'

# Create directories if they do not exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# List all image files
image_files = [f for f in os.listdir(original_data_dir) if os.path.isfile(os.path.join(original_data_dir, f))]
#image_files = [
    #f
   # for f in os.listdir(original_data_dir)
    #if os.path.isfile(os.path.join(original_data_dir, f))
    #and f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')) # Filter for image files
#]
# Print the number of images found to help debug
print("Number of images found:", len(image_files))
# Split the data
train_files, test_files = train_test_split(image_files, test_size=0.3, random_state=42)
val_files, test_files = train_test_split(test_files, test_size=0.5, random_state=42)  # Split remaining into val and test

# Helper function to copy files to destination
def copy_files(files, src_dir, dst_dir):
    for file in files:
        src_path = os.path.join(src_dir, file)
        dst_path = os.path.join(dst_dir, file)
        shutil.copy(src_path, dst_path)

# Copy the files to their respective directories
copy_files(train_files, original_data_dir, train_dir)
copy_files(val_files, original_data_dir, val_dir)
copy_files(test_files, original_data_dir, test_dir)

print(f"Training set: {len(train_files)} images")
print(f"Validation set: {len(val_files)} images")
print(f"Test set: {len(test_files)} images")

Number of images found: 78
Training set: 54 images
Validation set: 12 images
Test set: 12 images


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define directories
original_data_dir = '/content/new_arthritis/test/Normal'
train_dir = 'data/train'
val_dir = 'data/val'
test_dir = 'data/test'

# Create directories if they do not exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# List all image files
image_files = [f for f in os.listdir(original_data_dir) if os.path.isfile(os.path.join(original_data_dir, f))]

# Split the data
train_files, test_files = train_test_split(image_files, test_size=0.3, random_state=42)
val_files, test_files = train_test_split(test_files, test_size=0.5, random_state=42)  # Split remaining into val and test

# Helper function to copy files to destination
def copy_files(files, src_dir, dst_dir):
    for file in files:
        src_path = os.path.join(src_dir, file)
        dst_path = os.path.join(dst_dir, file)
        shutil.copy(src_path, dst_path)

# Copy the files to their respective directories
copy_files(train_files, original_data_dir, train_dir)
copy_files(val_files, original_data_dir, val_dir)
copy_files(test_files, original_data_dir, test_dir)

print(f"Training set: {len(train_files)} images")
print(f"Validation set: {len(val_files)} images")
print(f"Test set: {len(test_files)} images")

Training set: 53 images
Validation set: 12 images
Test set: 12 images


In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Conv2D, Multiply, Add
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define the attention block
def attention_block(inputs):
    attention = Conv2D(1, kernel_size=1, activation='sigmoid')(inputs)
    return Multiply()([inputs, attention])

# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Add attention block and new top layers
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = attention_block(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)  # Assuming binary classification

model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define directories for training, validation, and test data
train_dir = '/content/new_arthritis/train'
val_dir = '/content/new_arthritis/val'
test_dir = '/content/new_arthritis/test'

# Create data generators
train_datagen = ImageDataGenerator(rescale=0.2)
val_datagen = ImageDataGenerator(rescale=0.2)
test_datagen = ImageDataGenerator(rescale=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)
model.summary()
# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

# Save the model
model.save('resnet_with_attention.h5')

Found 762 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 154 images belonging to 2 classes.


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resnet50 (Functional)     │ (None, 7, 7, 2048)     │     23,587,712 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 7, 7, 1)        │          2,049 │ resnet50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply_1 (Multiply)     │ (None, 7, 7, 2048)     │              0 │ resnet50[0][0],        │
│                           │                        │                │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 2048)           │              0 │ multiply_1[0][0]       │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 256)            │        524,544 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 1)              │            257 │ dense_2[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 24,114,562 (91.99 MB)

 Trainable params: 526,850 (2.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


24/24 ━━━━━━━━━━━━━━━━━━━━ 21s 441ms/step - accuracy: 0.9257 - loss: 0.1747 - val_accuracy: 1.0000 - val_loss: 0.0041
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 131ms/step - accuracy: 1.0000 - loss: 0.0030 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 135ms/step - accuracy: 1.0000 - loss: 7.8843e-04 - val_accuracy: 1.0000 - val_loss: 2.6508e-04
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 159ms/step - accuracy: 1.0000 - loss: 3.8852e-04 - val_accuracy: 1.0000 - val_loss: 9.5544e-05
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 1.0000 - loss: 1.7947e-04 - val_accuracy: 1.0000 - val_loss: 6.6153e-05
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 6s 142ms/step - accuracy: 1.0000 - loss: 1.0847e-04 - val_accuracy: 1.0000 - val_loss: 6.9610e-05
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 112ms/step - accuracy: 1.0000 - loss: 1.1478e-04 - val_accuracy: 1.0000 - val_loss: 6.2020e-05
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 111ms/step - accuracy: 1.0000 - 

Test accuracy: 0.9935064911842346


In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resnet50 (Functional)     │ (None, 7, 7, 2048)     │     23,587,712 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 7, 7, 1)        │          2,049 │ resnet50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply (Multiply)       │ (None, 7, 7, 2048)     │              0 │ resnet50[0][0],        │
│                           │                        │                │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 2048)           │              0 │ multiply[0][0]         │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 256)            │        524,544 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │            257 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 25,168,264 (96.01 MB)

 Trainable params: 526,850 (2.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 1,053,702 (4.02 MB)

In [ ]:
import tensorflow as tf

def get_gradcam_heatmap(model, img_array, layer_name):
    # Get the model's output and gradients
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]  # Assuming binary classification and predicting class 1 (arthritis)

    # Compute gradients of the loss with respect to the conv outputs
    grads = tape.gradient(loss, conv_outputs)

    # Mean of gradients for each feature map
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))

    # Multiply each channel in the feature map by the corresponding mean gradient
    conv_outputs = conv_outputs[0]  # Remove the batch dimension
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.reduce_mean(heatmap, axis=-1)  # Mean across channels

    # Apply ReLU to the heatmap
    heatmap = tf.maximum(heatmap, 0)
    heatmap /= tf.reduce_max(heatmap)  # Normalize to [0, 1]

    return heatmap.numpy()

In [ ]:
from tensorflow.keras.preprocessing import image  # Import the 'image' module from keras

img_path = '/content/drive/MyDrive/MCP_arthritis/48167-Afbeelding1.jpg'  # Update this path
img = image.load_img(img_path, target_size=(224, 224))  # Resize according to your model's input size
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize if your model expects it

In [ ]:
# Specify the layer name for which you want to generate the heatmap
layer_name = 'conv2d_1'  # Change this to the last convolutional layer in your model

# Get the Grad-CAM heatmap
heatmap = get_gradcam_heatmap(model, img_array, layer_name)

In [ ]:
# Resize heatmap to the original image size
heatmap = np.expand_dims(heatmap, axis=-1)
heatmap_resized = tf.image.resize(heatmap, (img.size[1], img.size[0]))  # img.size returns (width, height)

# Convert the heatmap to RGB
heatmap_colored = plt.cm.jet(heatmap_resized.numpy()[:, :, 0])  # Use a color map for better visualization
heatmap_colored = heatmap_colored[:, :, :3]  # Remove the alpha channel

# Overlay the heatmap on the original image
superimposed_img = heatmap_colored * 0.4 + img_array[0]  # Adjust alpha as necessary

# Display the heatmap and the superimposed image
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(heatmap_resized[:, :, 0], cmap='jet')
plt.title('Heatmap')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(superimposed_img)
plt.title('Superimposed Image')
plt.axis('off')
plt.show()